In [1]:
import torch
from torchtext import data
from torchtext import datasets
import random
import torch.nn as nn
import torch.optim as optim
import time

TEXT = data.Field(tokenize = 'spacy')
LABEL = data.LabelField(dtype = torch.float)
torch.backends.cudnn.deterministic = True

In [2]:
train, test = datasets.IMDB.splits(TEXT, LABEL)


In [3]:
print('Number of training examples:', len(train))
print('Number of testing examples:', len(test))

Number of training examples: 25000
Number of testing examples: 25000


In [4]:
train, valid = train.split()

In [5]:
print('Number of training examples:', len(train))
print('Number of validation examples:', len(valid))
print('Number of testing examples:', len(test))

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [6]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train)

In [7]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 202591), (',', 193359), ('.', 166024), ('a', 109883), ('and', 109546), ('of', 100794), ('to', 93760), ('is', 76317), ('in', 61380), ('I', 54386), ('it', 53792), ('that', 49476), ('"', 44630), ("'s", 43688), ('this', 42380), ('-', 37459), ('/><br', 35528), ('was', 34879), ('as', 30256), ('with', 30070)]


In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test), 
    batch_size = 64,
    device = device)

In [10]:
class RNN(nn.Module):
    def __init__(self, in_dim, emb_dim, h_dim, out_dim):
        super().__init__()
        self.embedding = nn.Embedding(in_dim, emb_dim)
        
        self.rnn = nn.RNN(emb_dim, h_dim)
        
        self.fc = nn.Linear(h_dim, out_dim)
        
    def forward(self, text):
        
        embedded = self.embedding(text)
        
        output, hidden = self.rnn(embedded)
        
        return self.fc(hidden.squeeze(0))

In [11]:
in_dim = len(TEXT.vocab)
emb_dim = 100
h_dim = 256
out_dim = 1

model = RNN(in_dim, emb_dim, h_dim, out_dim)

In [12]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print('The model has {} trainable parameters'.format(count_parameters(model)))

The model has 2592105 trainable parameters


In [13]:
optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [14]:
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

In [15]:
def binary_accuracy(predictions, y):
    rounded = torch.round(torch.sigmoid(predictions))
    correct = (rounded == y).float() 
    acc = correct.sum() / len(correct)
    return acc

In [16]:
def Train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [17]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [18]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [20]:
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_t = time.time()
    
    train_loss, train_acc = Train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_t = time.time()

    epoch_mins, epoch_secs = epoch_time(start_t, end_t)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'FirstModel.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 6m 16s
	Train Loss: 0.693 | Train Acc: 49.40%
	 Val. Loss: 0.694 |  Val. Acc: 49.90%
Epoch: 02 | Epoch Time: 6m 18s
	Train Loss: 0.693 | Train Acc: 49.56%
	 Val. Loss: 0.694 |  Val. Acc: 50.57%
Epoch: 03 | Epoch Time: 6m 37s
	Train Loss: 0.693 | Train Acc: 49.98%
	 Val. Loss: 0.694 |  Val. Acc: 50.11%
Epoch: 04 | Epoch Time: 7m 11s
	Train Loss: 0.693 | Train Acc: 50.44%
	 Val. Loss: 0.694 |  Val. Acc: 49.87%
Epoch: 05 | Epoch Time: 7m 28s
	Train Loss: 0.693 | Train Acc: 49.72%
	 Val. Loss: 0.694 |  Val. Acc: 50.19%


In [21]:
model.load_state_dict(torch.load('FirstModel.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.707 | Test Acc: 42.25%
